<a href="https://colab.research.google.com/github/nihalgunu/DistractedDriverDirector-DDD/blob/master/DistractedDriverRecognitionApplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip CleanedDrivingSet.zip

Archive:  CleanedDrivingSet.zip
replace __MACOSX/._DrivingSet? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
img_height, img_width = 64, 64
batch_size = 100

train_ds = tf.keras.utils.image_dataset_from_directory(
    "DrivingSet/train",
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Found 20106 files belonging to 7 classes.


In [ ]:
class_names = ["Safe_Driving", "Cellphone_Usage", "Hand_On_Face", "Drinking", "Turned_Away", "One_Hand_On_Wheel", "Eyes_Off_Road"]
plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3,3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
model = tf.keras.Sequential(
    [
      tf.keras.layers.Rescaling(1./255),
      tf.keras.layers.Conv2D(128, 3, activation="relu"),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(128, 3, activation = "relu"),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation ="relu"),
      tf.keras.layers.Dense(7)
    ]

)

In [ ]:
model.compile(
    optimizer="adam",
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train_ds,
    epochs = 4
)

Epoch 1/4
202/202 [==============================] - 562s 3s/step - loss: 0.9006 - accuracy: 0.6808
Epoch 2/4
202/202 [==============================] - 549s 3s/step - loss: 0.1335 - accuracy: 0.9615
Epoch 3/4
202/202 [==============================] - 543s 3s/step - loss: 0.0495 - accuracy: 0.9859
Epoch 4/4
202/202 [==============================] - 560s 3s/step - loss: 0.0219 - accuracy: 0.9938


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite2", 'wb') as f:
  f.write(tflite_model)